## [Cartopy官方文档](https://scitools.org.uk/cartopy/docs/latest/)
### cartopy的地图投影位于cartopy.crs中 [cartopy投影文档](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html)
### import cartopy.feature as cfeature
1. cfeature自带[Example](https://scitools.org.uk/cartopy/docs/latest/gallery/lines_and_polygons/features.html#sphx-glr-gallery-lines-and-polygons-features-py)
2. NaturalEarthFeature[API reference](https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.feature.NaturalEarthFeature.html)
3. ShapelyFeature[]()


In [7]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.io.shapereader import Reader, natural_earth

plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False

## 读取数据
### method1 sg+double logistic function+derive 2rd

In [8]:
def read_data_nc1(yr):
    global lat, lon
    inpath = ("E:\Gosif_CHN\phenology\CHN_dbl_derive_%d.nc" % yr)
    with nc.Dataset(inpath, mode='r') as f:
        # print(f.variables.keys())
        # print(f.variables['sif'])
        SOP = (f.variables['SOP2'][:])
        lat = (f.variables['lat'][:])
        lon = (f.variables['lon'][:])

        SOP[SOP == -9999] = np.nan
    return SOP

### method2 sg+cubic+ratio2

In [9]:
def read_data_nc2(yr):
    global lat, lon
    inpath = ("E:\Gosif_CHN\phenology\CHN_cb_ratio2_%d.nc" % yr)
    with nc.Dataset(inpath, mode='r') as f:
        # print(f.variables.keys())
        # print(f.variables['sif'])
        SOP = (f.variables['SOP'][:])
        lat = (f.variables['lat'][:])
        lon = (f.variables['lon'][:])

        SOP[SOP == -9999] = np.nan
    return SOP

### method3 sg+cubic+seasonal amplitude

In [10]:
def read_data_nc3(yr):
    global lat, lon
    inpath = ("E:\Gosif_CHN\phenology\CHN_cb_samp_%d.nc" % yr)
    with nc.Dataset(inpath, mode='r') as f:
        # print(f.variables.keys())
        # print(f.variables['sif'])
        SOP = (f.variables['SOP'][:])
        lat = (f.variables['lat'][:])
        lon = (f.variables['lon'][:])

        SOP[SOP == -9999] = np.nan
    return SOP

### 读取LUC数据 Glass-glc 2015数据

In [11]:
def read_data_lcc():
    inpath = r"E:\GLASS-glc\GLASS-GLC_2015_GOSIF_nearest_CHN.nc"
    with nc.Dataset(inpath, mode='r') as f:
        lcc = (f.variables['lcc'][:])
    return lcc

In [30]:
'''Cartopy 绘图'''
def CreatMap(data, lon, lat, levels, cmap, savename):
    fig = plt.figure(figsize=(14, 6), dpi=1080)
    proj = ccrs.PlateCarree()
    ax = fig.add_subplot(111,projection = proj)
    fig.subplots_adjust(left=0.1, bottom=0.2, right=0.9, top=0.9, wspace=None, hspace=None)
    
    #区域
    #region = [-18, 52, -35, 37]
    region = [70, 140, 15, 55]
    ax.set_extent(region, crs=ccrs.PlateCarree())
    
    # 打开边框
    ax.spines['geo'].set_visible(True)
    
    #设置cfeature自带底图
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1)
    
    #设置shp文件
    shp_path1 = r'E:\SHP\gadm36_CHN_shp\gadm36_CHN_1.shp'
    shp_path2 = r'E:/SHP/world_shp/world_adm0_Project.shp'
    
    provinces = cfeature.ShapelyFeature(
        Reader(shp_path1).geometries(),
        ccrs.PlateCarree(), 
        edgecolor='k',
        facecolor='none')
    
    world = cfeature.ShapelyFeature(
        Reader(shp_path2).geometries(),
        ccrs.PlateCarree(), 
        edgecolor='k',
        facecolor='none')
    ax.add_feature(provinces, linewidth=0.8, zorder=3)
    ax.add_feature(world, linewidth=0.8, zorder=3)
    
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())

    ax.set_xticks(list(range(70, 141, 10)))
    ax.set_yticks(list(range(20, 51, 10)))#需要显示的纬度
    ax.tick_params(labelsize= 15)
    
    # 设置网格点
    lb=ax.gridlines(draw_labels=True, x_inline=False, y_inline=False,\
            linewidth=0.5, color='lightgray', linestyle='--' , alpha=0.8)# alpha是透明度
    
    lb.top_labels= False
    lb.bottom_labels= False
    lb.right_labels= False
    lb.left_labels= False
    lb.xlabel_style={'size':15}#修改经纬度字体大小
    lb.ylabel_style={'size':15}
    
    
    
    #绘制填色图等
    lon, lat = np.meshgrid(lon, lat)
    cs=ax.contourf(lon, lat, data, \
                   levels=levels, transform=ccrs.PlateCarree(), \
                   cmap=cmap, extend="both", zorder=1)
    
    cs2 = ax.contourf(lon, lat, lcc,
                      levels=[5, 15], transform=ccrs.PlateCarree(),
                      colors="white", extend="neither", zorder=2)
    
    cbar_ax = fig.add_axes([0.25, 0.1, 0.5, 0.03])
    cb = fig.colorbar(cs, cax=cbar_ax, orientation='horizontal')
    cb.ax.tick_params(labelsize=None)
    cb.set_label('days of year', fontsize=15)
    
    
    plt.suptitle(str(savename), fontsize=20)
    #plt.savefig(r'D:/Gosif_CHN/物候2001-2021/dbl_derive2/%s.jpg' %savename)
    #plt.savefig(r'D:/Gosif_CHN/物候2001-2021/cb_ratio2/%s.jpg' %savename)
    #plt.savefig(r'D:/Gosif_CHN/物候2001-2021/三种方法比较/%s.jpg' %savename)
    plt.show()

## 画图（每种方法单独绘图）
### 01-21基本态画图

In [ ]:
for yr in range(2001, 2002):
    sop = read_data_nc3(yr)
    lcc = read_data_lcc()
    sop[lcc == 10] = np.nan
    print('---------start of the photosynthetic period---------')
    print('%d SOP 95 percentile is: ' % yr, np.nanpercentile(sop, 90))
    print('%d SOP 5 percentile is: ' % yr, np.nanpercentile(sop, 10))

    lev1 = np.arange(95, 181, 5)
    cmap = 'rainbow_r'
    CreatMap(sop, lon, lat, lev1, cmap, savename="SOP %d" % (yr))

### 01-02三种方法分别距平

In [ ]:
for yr in range(2001, 2022):
    sop=read_data_nc1(yr)
    sop=sop.reshape(1, 800, 1400)
    if yr == 2001:
        sop_all= sop
    else:
        sop_all= np.vstack((sop_all, sop))
            
sop_ave=np.mean(sop_all, axis=0)
lcc = read_data_lcc()
for yr in range(2001, 2002):
    sop=read_data_nc1(yr)
    sop_d=sop-sop_ave
    sop_d[lcc == 10] = np.nan
    print('---------start of the photosynthetic period anomaly---------')
    print('%d SOP anomaly 95 percentile is: ' % yr, np.nanpercentile(sop_d, 90))
    print('%d SOP anomaly 5 percentile is: ' % yr, np.nanpercentile(sop_d, 10))
    
    lev1 = np.arange(-16, 17, 2)
    cmap = 'RdBu_r'
    CreatMap(sop_d, lon, lat, lev1, cmap, savename="SOP anomaly %d" % (yr))

### 平均值

In [ ]:
for yr in range(2001, 2022):
    sop=read_data_nc1(yr)
    sop=sop.reshape(1, 800, 1400)
    if yr == 2001:
        sop_all= sop
    else:
        sop_all= np.vstack((sop_all, sop))
            
sop_ave=np.mean(sop_all, axis=0)

## 三种方法比较
### 三种方法数据标准差

In [9]:
lcc = read_data_lcc()
def data_all(i):
    for yr in range(2001, 2022):
        sop = eval("read_data_nc%d(%d)" %(i, yr))
        sop = sop.reshape(1, 800, 1400)
        if yr == 2001:
            sop_all = sop
        else:
            sop_all = np.vstack((sop_all, sop))
    return sop_all

sop1_all = data_all(1)
sop2_all = data_all(2)
sop3_all = data_all(3)

for yr in range(21):
    a=sop1_all[yr, :, :].reshape(1, 800, 1400)
    b=sop2_all[yr, :, :].reshape(1, 800, 1400)
    c=sop3_all[yr, :, :].reshape(1, 800, 1400)
    sop_yr=np.vstack((a, b, c))
    del a, b, c
    
    sop_yr_std=np.std(sop_yr, axis=0).data
    sop_yr_std[lcc==10]=np.nan
    
    print('---------start of the photosynthetic period std---------')
    print('%d SOP year std 90 percentile is: ' % yr, np.nanpercentile(sop_yr_std, 90))
    print('%d SOP year std 10 percentile is: ' % yr, np.nanpercentile(sop_yr_std, 10))
    
    lev_l=int(np.nanpercentile(sop_yr_std, 10))
    lev_r=math.ceil(np.nanpercentile(sop_yr_std, 90))
    lev1=np.arange(lev_l, lev_r, 1)
    cmap="Blues"
    
    CreatMap(sop_yr_std, lon, lat, lev1, cmap, savename="SOP 3 method std %d" % (yr+2001))

'\nprint(\'---------start of the photosynthetic period---------\')\nprint(\'%d SOP 95 percentile is: \' % yr, np.nanpercentile(sop, 90))\nprint(\'%d SOP 5 percentile is: \' % yr, np.nanpercentile(sop, 10))\n\nlev1 = np.arange(95, 181, 5)\ncmap = \'rainbow_r\'\n# CreatMap(sop, lon, lat, lev1, cmap, savename="SOP %d" % (yr))\n'

### 三种方法距平后标准差